<a href="https://colab.research.google.com/github/swapnilsethi/Stat-5000/blob/main/Mahalanobis_Distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [330]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns
pd.pandas.set_option('display.max_columns',None) #to display all columns in output
np.set_printoptions(threshold=sys.maxsize)
from scipy.stats import chi2
from matplotlib import patches
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import math
from yellowbrick.regressor import residuals_plot
from google.colab import files
from yellowbrick.regressor import prediction_error

In [331]:
#read train data from github
URL = "https://raw.githubusercontent.com/swapnilsethi/Stat-5000/master/X_train-2.csv"
#data = pd.read_csv('train.csv')
data = pd.read_csv(URL)

In [332]:
Feature_kept_for_replacement_detection = [feature for feature in data if '_nan' in feature]
data = data.drop(['Unnamed: 0'], axis =1 )
data = data.drop(Feature_kept_for_replacement_detection, axis =1 )
data

,Id,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,1,208500.0,0.235294,0.666667,0.207668,0.033420,1.0,0.000000,0.333333,1.0,0.00,0.0,0.6875,0.666667,1.0,1.0,1.0,0.666667,0.500,0.043796,0.112903,0.0,0.0,0.857143,0.857143,0.666667,0.666667,1.0,1.000000,0.75,0.75,0.25,1.000000,0.125089,0.8,0.064212,0.140098,1.0,1.00,1.0,1.0,0.109641,0.413559,0.259231,0.333333,0.50,0.375,0.333333,0.50,0.461538,1.000000,0.00,0.00,0.75,0.092760,0.666667,0.4,0.368280,0.666667,1.0,1.0,0.000000,0.082210,0.090909,0.50,0.666667,0.666667
1,2,181500.0,0.000000,0.666667,0.255591,0.038795,1.0,0.000000,0.333333,1.0,0.25,0.0,0.5000,0.333333,1.0,1.0,0.8,0.555556,0.875,0.233577,0.532258,0.0,0.0,0.142857,0.142857,0.000000,0.333333,1.0,0.666667,0.75,0.75,1.00,0.666667,0.173281,0.8,0.121575,0.206547,1.0,1.00,1.0,1.0,0.194917,0.000000,0.174830,0.000000,0.50,0.375,0.333333,0.25,0.307692,1.000000,0.25,0.50,0.75,0.104525,0.666667,0.4,0.309140,0.666667,1.0,1.0,0.209270,0.000000,0.363636,0.25,0.666667,0.666667
2,3,223500.0,0.235294,0.666667,0.217252,0.046507,1.0,0.333333,0.333333,1.0,0.00,0.0,0.6875,0.666667,1.0,1.0,1.0,0.666667,0.500,0.058394,0.129032,0.0,0.0,0.857143,0.857143,0.666667,0.666667,1.0,1.000000,0.75,0.75,0.50,1.000000,0.086109,0.8,0.185788,0.150573,1.0,1.00,1.0,1.0,0.123083,0.419370,0.273549,0.333333,0.50,0.375,0.333333,0.50,0.307692,1.000000,0.25,0.50,0.75,0.093665,0.666667,0.4,0.408602,0.666667,1.0,1.0,0.000000,0.056604,0.727273,0.50,0.666667,0.666667
3,4,140000.0,0.294118,0.666667,0.191693,0.038561,1.0,0.333333,0.333333,1.0,0.50,0.0,0.7500,0.666667,1.0,1.0,1.0,0.666667,0.500,0.671533,0.612903,0.0,0.0,0.285714,0.285714,0.000000,0.333333,1.0,0.333333,0.50,1.00,0.25,0.666667,0.038271,0.8,0.231164,0.123732,1.0,0.75,1.0,1.0,0.131695,0.366102,0.260550,0.333333,0.25,0.375,0.333333,0.50,0.384615,1.000000,0.25,0.75,0.25,0.094118,0.333333,0.6,0.431452,0.666667,1.0,1.0,0.000000,0.047170,0.090909,0.00,0.666667,0.000000
4,5,250000.0,0.235294,0.666667,0.268371,0.060576,1.0,0.333333,0.333333,1.0,0.25,0.0,1.0000,0.666667,1.0,1.0,1.0,0.777778,0.500,0.065693,0.161290,0.0,0.0,0.857143,0.857143,0.666667,0.666667,1.0,1.000000,0.75,0.75,0.75,1.000000,0.116052,0.8,0.209760,0.187398,1.0,1.00,1.0,1.0,0.170342,0.509927,0.351168,0.333333,0.50,0.500,0.333333,0.50,0.538462,1.000000,0.25,0.50,0.75,0.094118,0.666667,0.6,0.561828,0.666667,1.0,1.0,0.134831,0.113208,1.000000,0.50,0.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,175000.0,0.235294,0.666667,0.198083,0.030929,1.0,0.000000,0.333333,1.0,0.00,0.0,0.6250,0.666667,1.0,1.0,1.0,0.555556,0.500,0.065693,0.145161,0.0,0.0,0.857143,0.857143,0.000000,0.333333,1.0,1.000000,0.75,0.75,0.25,0.833333,0.000000,0.8,0.407962,0.155974,1.0,1.00,1.0,1.0,0.130015,0.336077,0.247362,0.000000,0.50,0.375,0.333333,0.25,0.384615,1.000000,0.25,0.50,0.75,0.094118,0.666667,0.4,0.309140,0.666667,1.0,1.0,0.000000,0.053908,0.636364,0.25,0.666667,0.666667
1456,1457,210000.0,0.000000,0.666667,0.271565,0.055505,1.0,0.000000,0.333333,1.0,0.00,0.0,0.5625,0.666667,1.0,1.0,0.8,0.555556,0.625,0.240876,0.387097,0.0,0.0,0.571429,0.714286,1.000000,0.333333,1.0,0.666667,0.7

In [333]:
def mahalanobis_distance(box_data):
    box_data = box_data.drop(['Id'], axis=1)
    box_data = box_data.to_numpy()
    # Covariance matrix
    covariance  = np.cov(box_data , rowvar=False)

    # Covariance matrix power of -1
    covariance_pm1 = np.linalg.matrix_power(covariance, -1)

    # Center point
    centerpoint = np.mean(box_data , axis=0)

        # Distances between center point and other points
    distances = []
    for i, val in enumerate(box_data):
        p1 = val
        p2 = centerpoint
        distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
        distances.append(distance)
    distances = np.array(distances)
   
    
    # Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outliers 
    cutoff = chi2.ppf(0.5, box_data.shape[1])
    print(cutoff)
    # Index of outliers
    outlierIndexes = np.where(distances > cutoff )

    print('--- Index of Outliers ----')
    print(outlierIndexes)

    #print(cutoff.dtype)
    #print(distances.dtype)
    #print('--- Observations found as outlier -----')
    #print(box_data[ distances > cutoff , :])
    return outlierIndexes, distances


In [334]:
temp_data = data.drop('SalePrice', axis=1)
data_outlierIndexes, distances = mahalanobis_distance(temp_data)

64.33456259846689
--- Index of Outliers ----
(array([   3,    5,    7,    8,    9,   11,   17,   21,   24,   28,   29,
         30,   39,   41,   42,   43,   46,   48,   51,   52,   53,   54,
         58,   61,   63,   66,   69,   74,   78,   81,   88,   90,   92,
         93,   95,   97,   98,  102,  104,  106,  108,  109,  113,  114,
        118,  120,  125,  127,  129,  142,  144,  146,  149,  153,  154,
        155,  156,  165,  166,  169,  170,  171,  172,  174,  175,  177,
        178,  179,  181,  182,  183,  184,  185,  187,  188,  190,  197,
        198,  202,  206,  210,  217,  218,  233,  242,  246,  249,  250,
        251,  253,  259,  260,  263,  264,  267,  268,  269,  271,  275,
        291,  292,  300,  304,  307,  312,  313,  314,  322,  323,  324,
        325,  328,  329,  330,  332,  335,  341,  342,  344,  351,  352,
        355,  358,  362,  365,  367,  371,  373,  375,  378,  383,  384,
        386,  390,  393,  398,  400,  406,  410,  411,  412,  414,  417,
     

In [335]:
distances = (distances.astype(int))
print(distances)
extreme_out_lier = np.where(distances>200)
#extreme_out_lier

[  17   51   18   88   21   80   23   74  174  216   30   66   52   22
   37   48   43   71   36   52   32   99   21   44   68   26   40   27
   67  175  119   35   34   54   38   31   39   43   52  136   49   66
   67   67   57   45   82   29  172   27   38   91  363  180   77   25
   52   15   93   41   41   79   33  161   43   28   65   14   50  105
   54   33   25   48  129   57   38   59   85   46   57   66   32   48
   29   36   50   58  302   45   75   52   78  147   30   79   24   94
   99   58   41   26  106   29  124   33   91   53  143   64   50   33
   37  114   70   46   61   38   74   25  150   45   30   22   52  135
   39   68   35   84   39   24   33   31   38   30   28   62   35   22
   61   27  100   18   82   51   64   27   54   70   36   35   30  101
   88  118   92   38   27   62   35   34   40   49   57   80  152   42
   13  122  161   88   78   51   83   80   43   70   76   73   43   77
  105   92  181  297   41  124  109   45  108   53   23   50   33   32
   36 

In [336]:
'''
## Finding ellipse dimensions 
pearson = covariance[0, 1]/np.sqrt(covariance[0, 0] * covariance[1, 1])
ell_radius_x = np.sqrt(1 + pearson)
ell_radius_y = np.sqrt(1 - pearson)
lambda_, v = np.linalg.eig(covariance)
lambda_ = np.sqrt(lambda_)

# Ellipse patch
ellipse = patches.Ellipse(xy=(centerpoint[0], centerpoint[1]),
                  width=lambda_[0]*np.sqrt(cutoff)*2, height=lambda_[1]*np.sqrt(cutoff)*2,
                  angle=np.rad2deg(np.arccos(v[0, 0])), edgecolor='#fab1a0')
ellipse.set_facecolor('#0984e3')
ellipse.set_alpha(0.5)
fig = plt.figure()
ax = plt.subplot()
ax.add_artist(ellipse)
plt.scatter(box_data[: , 0], box_data[ : , 1])
plt.show()
'''

"\n## Finding ellipse dimensions \npearson = covariance[0, 1]/np.sqrt(covariance[0, 0] * covariance[1, 1])\nell_radius_x = np.sqrt(1 + pearson)\nell_radius_y = np.sqrt(1 - pearson)\nlambda_, v = np.linalg.eig(covariance)\nlambda_ = np.sqrt(lambda_)\n\n# Ellipse patch\nellipse = patches.Ellipse(xy=(centerpoint[0], centerpoint[1]),\n                  width=lambda_[0]*np.sqrt(cutoff)*2, height=lambda_[1]*np.sqrt(cutoff)*2,\n                  angle=np.rad2deg(np.arccos(v[0, 0])), edgecolor='#fab1a0')\nellipse.set_facecolor('#0984e3')\nellipse.set_alpha(0.5)\nfig = plt.figure()\nax = plt.subplot()\nax.add_artist(ellipse)\nplt.scatter(box_data[: , 0], box_data[ : , 1])\nplt.show()\n"

In [337]:
data_outlier_data = data.iloc[np.asarray(data_outlierIndexes).flatten()]
data_outlier_data = data_outlier_data.drop(np.asarray(extreme_out_lier).flatten())

In [338]:
#sns.pairplot(data = data_outlier_data)
#from google.colab import files
#plt.savefig("Pairplot.png")
#files.download("Pairplot.png") 

In [339]:
data_non_outlier_data = data.drop(np.asarray(data_outlierIndexes).flatten())

In [340]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mse = sklearn.metrics.mean_squared_error(np.log(test_labels),np.log(predictions))
    RMSE = math.sqrt(mse)
    print('Model Performance')
    print('RMSE: {:0.4f} degrees.'.format(RMSE))
    return RMSE

In [341]:
def xgboost(data):
    X = data.drop(['Id','SalePrice'],1)
    Y = (data['SalePrice']).to_frame()
    X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
    Y = Y.to_numpy()
    y_train= y_train.to_numpy()
    y_test = y_test.to_numpy()
    data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
    global xg_reg
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42, learning_rate=0.25, n_estimators = 100, max_depth=4)
    xg_reg.fit(X_train,y_train)
    preds = xg_reg.predict(X_test)
    XGB_accuracy = evaluate(xg_reg, X_test, y_test)

In [342]:
xgboost(data_non_outlier_data)

Model Performance
RMSE: 0.1123 degrees.


In [343]:
# Reading test data
URL = "https://raw.githubusercontent.com/swapnilsethi/Stat-5000/master/X_test-2.csv"
test_data = pd.read_csv(URL)
# Removing unwanted columns

Feature_kept_for_replacement_detection = [feature for feature in test_data if '_nan' in feature]
#print(Feature_kept_for_replacement_detection)
test_data = test_data.drop(Feature_kept_for_replacement_detection,1)
test_data= test_data.drop(['Unnamed: 0'], axis = 1)

# Making Predications

#Y_pred = xg_reg.predict(test_data)

In [ ]:
test_data_outlierIndexes, test_distances = mahalanobis_distance(test_data)
#test_data_outlierIndexes

In [345]:
test_data_outlier_data = test_data.iloc[np.asarray(test_data_outlierIndexes).flatten()]
test_data_outlier_data_Ids = test_data_outlier_data.iloc[:, 0].values 
test_data_non_outlier_data = test_data.drop(np.asarray(test_data_outlierIndexes).flatten())
test_data_non_outlier_data_Ids = test_data_non_outlier_data.iloc[:, 0].values

In [346]:
#test_data_outlier_data_Ids

In [347]:
test_data_non_outlier_data = test_data_non_outlier_data.drop('Id', axis=1)
Y_pred_non_outlier_data = xg_reg.predict(test_data_non_outlier_data)

**for Outlier data**

In [348]:
def xgboost_outlier(data):
    X = data.drop(['Id','SalePrice'],1)
    Y = (data['SalePrice']).to_frame()
    X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
    Y = Y.to_numpy()
    y_train= y_train.to_numpy()
    y_test = y_test.to_numpy()
    data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
    global xg_reg
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42, learning_rate=0.09,n_estimators = 110, max_depth=4)
    xg_reg.fit(X_train,y_train)
    preds = xg_reg.predict(X_test)
    XGB_accuracy = evaluate(xg_reg, X_test, y_test)

In [349]:
xgboost_outlier(data_outlier_data)

Model Performance
RMSE: 0.1480 degrees.


In [350]:
test_data_outlier_data = test_data_outlier_data.drop('Id', axis=1)
Y_pred_outlier_data = xg_reg.predict(test_data_outlier_data)

Combine Non_outlier_data_with_IDs

In [351]:
non_outlier_pred = pd.DataFrame(test_data_non_outlier_data_Ids, columns=['Id'])
non_outlier_pred['SalePrice'] = Y_pred_non_outlier_data.tolist()


In [309]:
outlier_pred = pd.DataFrame(test_data_outlier_data_Ids, columns=['Id'])
outlier_pred['SalePrice'] = Y_pred_outlier_data.tolist()

In [375]:
frames = [outlier_pred, non_outlier_pred]
pred = pd.concat(frames)

In [ ]:
#def Predications_file_generator(predications): 
    #pred = pd.DataFrame(ID, columns=['Id'])
    #pred['SalePrice'] = predications.tolist()

pred.to_csv('pred.csv')
files.download('pred.csv')

In [ ]:
#Predications_file_generator(Y_pred)

In [352]:
data_outlier_data = data.drop(np.asarray(extreme_out_lier).flatten())

In [369]:
def xgboost_outlier(data):
    X = data.drop(['Id','SalePrice'],1)
    Y = (data['SalePrice']).to_frame()
    X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
    Y = Y.to_numpy()
    y_train= y_train.to_numpy()
    y_test = y_test.to_numpy()
    data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
    global xg_reg
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42, learning_rate=0.27,n_estimators = 110, max_depth=4)
    xg_reg.fit(X_train,y_train)
    preds = xg_reg.predict(X_test)
    XGB_accuracy = evaluate(xg_reg, X_test, y_test)

In [370]:
xgboost_outlier(data_outlier_data)

Model Performance
RMSE: 0.1361 degrees.


In [371]:
test_data_outlier_data = test_data.drop('Id', axis=1)
Y_pred = xg_reg.predict(test_data_outlier_data)

In [372]:
test_data_Ids = test_data.iloc[:, 0].values 

In [373]:
pred = pd.DataFrame(test_data_Ids, columns=['Id'])
pred['SalePrice'] = Y_pred.tolist()

In [ ]:
pred.to_csv('pred.csv')
files.download('pred.csv')